In [ ]:
! pip install kaggle

In [ ]:
# configuring the path of json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

importing Twitter Sentiment dataset

In [ ]:
# API to fetch data set from kaggle
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment140


In [ ]:
# API to fetch data set from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 93% 75.0M/80.9M [00:00<00:00, 783MB/s]
100% 80.9M/80.9M [00:00<00:00, 776MB/s]


In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Processing

In [ ]:
# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [ ]:
# checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [ ]:
#printing the first 5 rows of the data frame
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# naming the columns and reading the dataset again
col_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=col_names, encoding='ISO-8859-1')

In [ ]:
# checking the number of rows and columns
twitter_data.shape

(1600000, 6)

In [ ]:
#printing the first 5 rows of the data frame
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# counting the number of missing valuse in the dataset
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
# cheking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Convert the target '4' to '1'

In [ ]:
twitter_data.replace({'target': {4: 1}}, inplace=True)
print("changed from 4 to 1")

changed from 4 to 1


In [ ]:
# cheking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 --> Negative Tweet

1 --> Positive Tweet

**Stemming**

Stemming is the process of reducing a word to its Root word

example: actor,acting,actress = act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')   # step-1


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

RAW = "/content/training.1600000.processed.noemoticon.csv"

# Assign column names manually (based on Sentiment140 format)
cols = ["target", "ids", "date", "flag", "user", "text"]
twitter_data = pd.read_csv(RAW, encoding="latin-1", names=cols, header=None)     # STEP-2


In [ ]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(language='english')


In [ ]:
import multiprocessing as mp

def stem_text(text):
    return " ".join([stemmer.stem(word) for word in str(text).split()])

# Create a pool of workers equal to your CPU core count
pool = mp.Pool(mp.cpu_count())

# Apply stemming in parallel on all tweets
twitter_data['stemmed_content'] = pool.map(stem_text, twitter_data['text']) #50 min approximately

pool.close()
pool.join()


In [ ]:
PROC_CSV = "/content/drive/MyDrive/processed_twitter.csv"
twitter_data.to_csv(PROC_CSV, index=False)

PROC_PARQUET = "/content/drive/MyDrive/processed_twitter.parquet"  #STEP-4
twitter_data.to_parquet(PROC_PARQUET, index=False)


In [ ]:
twitter_data.to_csv("/content/drive/MyDrive/processed_twitter.csv", index=False)


In [ ]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can't updat his facebook by t...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichan i dive mani time for the ball. manag...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, it not behav at all. i'm ..."


In [ ]:
print(twitter_data['stemmed_content'])

0          @switchfoot http://twitpic.com/2y1zl - awww, t...
1          is upset that he can't updat his facebook by t...
2          @kenichan i dive mani time for the ball. manag...
3               my whole bodi feel itchi and like it on fire
4          @nationwideclass no, it not behav at all. i'm ...
                                 ...                        
1599995    just woke up. have no school is the best feel ...
1599996    thewdb.com - veri cool to hear old walt interv...
1599997    are you readi for your mojo makeover? ask me f...
1599998    happi 38th birthday to my boo of alll time!!! ...
1599999    happi #charitytuesday @thenspcc @sparkschar @s...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    4
1599996    4
1599997    4
1599998    4
1599999    4
Name: target, Length: 1600000, dtype: int64


In [ ]:
# seperating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['@switchfoot http://twitpic.com/2y1zl - awww, that a bummer. you shoulda got david carr of third day to do it. ;d'
 "is upset that he can't updat his facebook by text it... and might cri as a result school today also. blah!"
 '@kenichan i dive mani time for the ball. manag to save 50% the rest go out of bound'
 ... 'are you readi for your mojo makeover? ask me for detail'
 'happi 38th birthday to my boo of alll time!!! tupac amaru shakur'
 'happi #charitytuesday @thenspcc @sparkschar @speakinguph4h']


In [ ]:
print(Y)

[0 0 0 ... 4 4 4]


Splitting the data to training data and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['about to watch saw iv and drink a lil wine' "@hatermagazin i'm in!"
 'even though it my favourit drink i think it the vodka and coke that wipe my mind all the time think im gonna have to find a new drink'
 ... 'is eager for monday afternoon'
 "hope everyon and their mother had a great day! can't wait to hear what the guy have in store tomorrow!"
 'i love wake up to folgers. too bad my voic was deeper than his.']


In [ ]:
print(X_test)

["@mmangen - m do fine! i haven't had much time to chat on twitter. hubbi is back for the summer &amp; tend to domin my free time."
 'at ah may show w/ ruth kim &amp; geoffrey sanhueza'
 '@ishatara mayb it was onli a bay area thang dammit!' ...
 '@destini41 nevertheless hooray! for 4700 member and have a wonder and safe trip!'
 'not feel too well' '@supersandro !!!! thank you!']


In [ ]:
# converting textual data to numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 15116263 stored elements and shape (1280000, 577937)>
  Coords	Values
  (0, 40367)	0.24921232209377872
  (0, 518339)	0.12516942437452067
  (0, 547839)	0.2582588486846576
  (0, 452411)	0.3392999122062133
  (0, 253881)	0.49751606063030857
  (0, 56992)	0.15416028092917775
  (0, 166661)	0.3586990798287864
  (0, 311521)	0.3994319150280691
  (0, 555059)	0.42532959332385556
  (1, 225963)	0.976511044044121
  (1, 246930)	0.21546735451074847
  (2, 518339)	0.07448629885867337
  (2, 56992)	0.09173828844231074
  (2, 166661)	0.4269120353308398
  (2, 184225)	0.17606492300327523
  (2, 513809)	0.17213476271015216
  (2, 252457)	0.17764992638374646
  (2, 365962)	0.18150122574208405
  (2, 190292)	0.27015799676026536
  (2, 512829)	0.2962591028468319
  (2, 507049)	0.15389013245484054
  (2, 544240)	0.3034847405014382
  (2, 129186)	0.28851899247541224
  (2, 506657)	0.1086812760282542
  (2, 555466)	0.3110525179977014
  :	:
  (1279998, 148646)	0.1830

In [ ]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3682654 stored elements and shape (320000, 577937)>
  Coords	Values
  (0, 55670)	0.1564127516080687
  (0, 74693)	0.14782115550554228
  (0, 117417)	0.24498793964771903
  (0, 161837)	0.13658475098480527
  (0, 162862)	0.3427759108350762
  (0, 193590)	0.23248044607150456
  (0, 197900)	0.10410024876512615
  (0, 200309)	0.21657520280838333
  (0, 222083)	0.15363662705357917
  (0, 226232)	0.2119589113655081
  (0, 238036)	0.26113570276760684
  (0, 251544)	0.10108053657453764
  (0, 354011)	0.41253989821942794
  (0, 363552)	0.16210049135056043
  (0, 365962)	0.09294381785221748
  (0, 390023)	0.11208506286559435
  (0, 491890)	0.20047695324987191
  (0, 504443)	0.31752691443802006
  (0, 507049)	0.07880462725062415
  (0, 515947)	0.2892464170881184
  (0, 518339)	0.15257287580953569
  (0, 529810)	0.16416924990603868
  (1, 45681)	0.29976374475708845
  (1, 55670)	0.20533399271736968
  (1, 68992)	0.1650684709426607
  :	:
  (319995, 525957)	0.377

Training the Machine Learning Model

**LOGISTIC** **REGRESSION**

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation and Accuracy Score

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.8200140625


In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', training_data_accuracy)

Accuracy score of the test data :  0.8200140625


Saving the trained model

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
import pickle

pickle.dump(vectorizer, open("tfidf_vectorizer.pkl", "wb"))  #saving fitted vectorizer and trained model
pickle.dump(model, open("sentiment_model.pkl", "wb"))


Using the saved model for future predictions

In [ ]:
# loading the saved model
loaded_model = pickle.load(open('trained_model.sav', 'rb'))

In [ ]:
twitter_data.replace({'target': {4: 1}}, inplace=True)
print("changed from 4 to 1")

changed from 4 to 1


In [ ]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print('The Tweet is Negative')
else:
  print('The Tweet is Positive')

4
[4]
The Tweet is Positive


In [ ]:
X_new = X_test[3]
print(Y_test[3])

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print('The Tweet is Negative')
else:
  print('The Tweet is Positive')

0
[0]
The Tweet is Negative


In [ ]:
vectorizer = pickle.load(open("tfidf_vectorizer.pkl", "rb"))
model = pickle.load(open("sentiment_model.pkl", "rb"))

tweet = "I hate this product"
X_new = vectorizer.transform([tweet])   # ✅ works because vectorizer is already fitted
print(model.predict(X_new))


[0]


In [ ]:
pip install gradio


In [ ]:
import gradio as gr
import matplotlib.pyplot as plt

def predict_sentiment(text):
    try:
        text_vector = vectorizer.transform([text])
        prediction = model.predict(text_vector)
        proba = model.predict_proba(text_vector)

        # get max confidence (positive or negative)
        max_confidence = max(proba[0]) * 100

        # Neutral case when confidence is between 50% and 60%
        if 50 <= max_confidence < 60:
            return f"<div style='color:orange; font-size:20px;'>😐 Neutral (Low confidence: {max_confidence:.2f}%)</div>"

        # Otherwise normal positive/negative
        if prediction[0] in [1, 4]:   # adjust according to dataset labels
            confidence = proba[0][1] * 100
            return f"<div style='color:green; font-size:20px;'>✅ Positive 😊 ({confidence:.2f}% confidence)</div>"
        else:
            confidence = proba[0][0] * 100
            return f"<div style='color:red; font-size:20px;'>❌ Negative 😞 ({confidence:.2f}% confidence)</div>"

    except Exception as e:
        return f"<div style='color:gray; font-size:16px;'>⚠️ Error: {str(e)}</div>"




def analyze_file(file):
    # Read file content
    with open(file.name, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # Sentiment counters
    pos, neg, neu = 0, 0, 0

    for line in lines:
        line = line.strip()
        if not line:
            continue
        result = predict_sentiment(line)

        if "Positive" in result:
            pos += 1
        elif "Negative" in result:
            neg += 1
        else:
            neu += 1

    total = pos + neg + neu
    if total == 0:
        return "No text found", None

    # Pie chart
    labels = ["Positive", "Negative", "Neutral"]
    sizes = [pos, neg, neu]
    colors = ["green", "red", "gray"]

    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, colors=colors, autopct="%1.1f%%", startangle=90)
    ax.axis("equal")  # Circle

    return f"✅ Analysis Completed:\nPositive: {pos}\nNegative: {neg}\nNeutral: {neu}", fig





In [ ]:
import gradio as gr

# --- UI 1: Single Text Analysis ---
text_interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=3, placeholder="Enter text here..."),
    outputs=gr.HTML(),
    title="🌟 Tweet Sentiment Analysis",
    description="Type a tweet or any text below and click **Analyze** to see if it's Positive, Negative or Neutral."
)

# --- UI 2: File Upload Analysis ---
file_interface = gr.Interface(
    fn=analyze_file,
    inputs=gr.File(type="filepath", file_types=[".txt"]),   # ✅ fixed here
    outputs=["text", "plot"],
    title="📂 Text File Sentiment Analysis",
    description="Upload a .txt file. The system will analyze each line and show sentiment distribution."
)

# --- Combine into Tabs ---
demo = gr.TabbedInterface([text_interface, file_interface],
                          ["Single Text", "File Analysis"])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e55e8f7051fefbe1f6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
